In [92]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


examples = [
    {
        "movie_title": "반지의 제왕",
        "answer": "💍🧙‍♂️🏔️",
    },
    {
        "movie_title": "매트릭스",
        "answer": "💾🕶️🤖",
    },
    {
        "movie_title": "쇼생크 탈출",
        "answer": "🪓🧱🚶‍♂️",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie_title}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 유명한 영화 평론가입니다. 대답은 이모티콘 세개로만 대답해주세요."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie_title}"),
    ]
)


def load_memory(_):
    # print(memory.load_memory_variables({})["history"])
    return memory.load_memory_variables({})["history"]



chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(movie_title):
    result = chain.invoke({"movie_title": movie_title})
    memory.save_context(
        {"input": movie_title},
        {"output": result.content},
    )
    



# chain.invoke({"movie_title": "택시"})

In [93]:
invoke_chain("포레스트 검프")
invoke_chain("타이타닉")

🌲🏌️‍♂️🏅🚢💔🌊

In [94]:
invoke_chain("앞서 물어본 마지막 두 영화 중 앞 영화")

🌲🏌️‍♂️🏅